In [141]:
import pandas as pd
import os
import re

In [142]:
# get the root directory, this notebook is in the notebooks directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_files_dir = os.path.join(root_dir, 'data', 'raw')

In [143]:
raw_files = os.listdir(raw_files_dir)
abs_raw_files = [os.path.join(raw_files_dir, f) for f in raw_files]

In [144]:
raw_transactions = pd.read_excel(abs_raw_files[0])

In [145]:
transactions

,Rekeningnummer,Muntsoort,Transactiedatum,Rentedatum,Beginsaldo,Eindsaldo,Transactiebedrag,TRTP,IBAN,NAME,REMI,EREF,ORDP,location
0,136006396,EUR,20240818,20240818,0.00,2.88,2.88,SEPA OVERBOEKING,NL13ABNA0506417344,Van A ESPINOZA ESPINA,Bezem,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED
1,136006396,EUR,20240818,20240818,2.88,2.87,-0.01,iDEAL,NL56DEUT0265186420,Swapfiets B.V.,13312242815179 8030118233868818 13312242815179,18-08-2024 20:28 8030118233868818,NOTPROVIDED,NOTPROVIDED
2,136006396,EUR,20240818,20240818,2.87,2.86,-0.01,iDEAL,NL56DEUT0265186420,NS Reizigers B.V.,VCR208110519023610 8030928857748188 R20-811051...,18-08-2024 20:36 8030928857748188,NOTPROVIDED,NOTPROVIDED
3,136006396,EUR,20240818,20240818,2.86,2.85,-0.01,iDEAL,NL56DEUT0265186420,NS Reizigers B.V.,VCR2083012768423734 8030609730868059 R20-83012...,18-08-2024 20:37 8030609730868059,NOTPROVIDED,NOTPROVIDED
4,136006396,EUR,20240818,20240818,2.85,9.85,7.00,SEPA OVERBOEKING,NL93BUNQ2036449662,Giedrius Mirklys,Transfer,NOTPROVIDED,ID,NOTPROVIDED
5,136006396,EUR,20240819,20240819,9.85,1009.85,1000.00,SEPA OVERBOEKING,LT637300010150874468,GIEDRIUS MIRKLYS,TRANSFER TO MY DUTCH ACCOUNT,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED
6,136006396,EUR,20240819,20240819,1009.85,986.50,-23.35,iDEAL,NL13ABNA0506417344,Espinoza,Jij dwong me,19-08-2024 09:02 0031129436745083,NOTPROVIDED,NOTPROVIDED
7,136006396,EUR,20240819,20240819,986.50,641.50,-345.00,SEPA OVERBOEKING,NL15RABO0105104329,Real Estate Malden B.V.,Huur september 2024,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED
8,136006396,EUR,20240819,20240819,641.50,1141.50,500.00,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED
9,136006396,EUR,20240819,20240819,1141.50,1141.88,0.38,SEPA OVERBOEKING,NL93BUNQ2036449662,Giedrius Mirklys,NOTPROVIDED,NOTPROVIDED,ID,NOTPROVIDED


In [146]:
raw_transactions["Omschrijving"][8]

'OUR REF:     TG1908003338203     ORIGINAL EUR              500,00RECEIVED AAB EUR          500,00 CHARGES SHARED                  /LT637300010150874468            GIEDRIUS MIRKLYS                Transfer to my Dutch account'

In [147]:
raw_transactions["Omschrijving"][12]

'BEA, Betaalpas                   ALBERT HEIJN 1015,PAS721        NR:MM060V, 19.08.24/14:13        NIJMEGEN'

In [148]:
raw_transactions["Omschrijving"][25].split("  ")

['BEA, Google Pay',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'BCK*Cafetaria Sema Hoo,PAS721',
 ' NR:CT812935, 21.08.24/20:25',
 '',
 '',
 'Nijmegen']

In [149]:
transactions = raw_transactions.copy()
def clean_list(lst):
    """Remove empty strings and strings with only spaces."""
    return [item for item in lst if item.strip()]

def parse_transaction(details):
    if details.startswith('/'):
        # Handle structured data
        parts = details.split('/')
        cleaned_parts = clean_list(parts)
        return dict(zip(cleaned_parts[::2], cleaned_parts[1::2]))
    else:
        # Handle unstructured data
        parts = details.split('  ')
        cleaned_parts = clean_list(parts)
        if cleaned_parts[0].startswith('BEA'):
            return {
                'TRTP': cleaned_parts[0],
                'NAME': cleaned_parts[1].split(',')[0].strip(),
                'IBAN': cleaned_parts[1].split(',')[1].strip(),
                'location': cleaned_parts[-1]
            }
    return {}

In [150]:
# Apply the parsing function to the 'Details' column
parsed_details = transactions["Omschrijving"].apply(parse_transaction)

# Convert the parsed details into a DataFrame and concatenate with the original DataFrame
parsed_df = pd.DataFrame(parsed_details.tolist())
transactions = pd.concat([transactions, parsed_df], axis=1)
transactions = transactions.drop(columns=['Omschrijving', "BIC", "EREF", "ORDP"])

In [151]:
transactions

,Rekeningnummer,Muntsoort,Transactiedatum,Rentedatum,Beginsaldo,Eindsaldo,Transactiebedrag,TRTP,IBAN,NAME,REMI,location
0,136006396,EUR,20240818,20240818,0.00,2.88,2.88,SEPA OVERBOEKING,NL13ABNA0506417344,AAB INZ TIKKIE,"Tikkie ID 000880523985, Bezem, Van A ESPINOZA ...",NaN
1,136006396,EUR,20240818,20240818,2.88,2.87,-0.01,iDEAL,NL56DEUT0265186420,Swapfiets B.V.,13312242815179 8030118233868818 13312242815179,NaN
2,136006396,EUR,20240818,20240818,2.87,2.86,-0.01,iDEAL,NL56DEUT0265186420,NS Reizigers B.V.,VCR208110519023610 8030928857748188 R20-811051...,NaN
3,136006396,EUR,20240818,20240818,2.86,2.85,-0.01,iDEAL,NL56DEUT0265186420,NS Reizigers B.V.,VCR2083012768423734 8030609730868059 R20-83012...,NaN
4,136006396,EUR,20240818,20240818,2.85,9.85,7.00,SEPA OVERBOEKING,NL93BUNQ2036449662,Giedrius Mirklys,Transfer,NaN
5,136006396,EUR,20240819,20240819,9.85,1009.85,1000.00,SEPA OVERBOEKING,LT637300010150874468,GIEDRIUS MIRKLYS,TRANSFER TO MY DUTCH ACCOUNT,NaN
6,136006396,EUR,20240819,20240819,1009.85,986.50,-23.35,iDEAL,NL13ABNA0506417344,Espinoza via Tikkie,000880676140 0031129436745083 Jij dwong me NL5...,NaN
7,136006396,EUR,20240819,20240819,986.50,641.50,-345.00,SEPA OVERBOEKING,NL15RABO0105104329,Real Estate Malden B.V.,Huur september 2024,NaN
8,136006396,EUR,20240819,20240819,641.50,1141.50,500.00,NaN,NaN,NaN,NaN,NaN
9,136006396,EUR,20240819,20240819,1141.50,1141.88,0.38,SEPA OVERBOEKING,NL93BUNQ2036449662,Giedrius Mirklys,NaN,NaN


In [152]:
transactions = transactions.fillna('NOTPROVIDED')

In [153]:
def update_name(row):
    # Check if 'tikkie' is in the NAME column (case insensitive)
    if 'tikkie' in row['NAME'].lower():
        if 'via' in row['NAME'].lower():
            # Remove 'via Tikkie' (case insensitive)
            row['NAME'] = re.sub(r'via\s+tikkie', '', row['NAME'], flags=re.IGNORECASE).strip()

            remi_row = row['REMI']
            
            pattern = r'\b\d+\b\s+\b\d+\b\s+(.*?)\s+([A-Z]{2}[A-Z0-9]+)'
            match = re.search(pattern, remi_row)
            if match:
                # The substring is captured in the first group
                substring = match.group(1)
                remi_row = substring.strip()
            else:
                remi_row = None

            row['REMI'] = remi_row
        else:
            # Split the REMI column by comma and take the second to last element
            remi_parts = [part.strip() for part in row['REMI'].split(',')]
            if len(remi_parts) > 1:
                row['NAME'] = remi_parts[-2]
                row['REMI'] = remi_parts[1]
    return row

In [154]:
transactions = transactions.apply(update_name, axis=1)

In [155]:
transactions

,Rekeningnummer,Muntsoort,Transactiedatum,Rentedatum,Beginsaldo,Eindsaldo,Transactiebedrag,TRTP,IBAN,NAME,REMI,location
0,136006396,EUR,20240818,20240818,0.00,2.88,2.88,SEPA OVERBOEKING,NL13ABNA0506417344,Van A ESPINOZA ESPINA,Bezem,NOTPROVIDED
1,136006396,EUR,20240818,20240818,2.88,2.87,-0.01,iDEAL,NL56DEUT0265186420,Swapfiets B.V.,13312242815179 8030118233868818 13312242815179,NOTPROVIDED
2,136006396,EUR,20240818,20240818,2.87,2.86,-0.01,iDEAL,NL56DEUT0265186420,NS Reizigers B.V.,VCR208110519023610 8030928857748188 R20-811051...,NOTPROVIDED
3,136006396,EUR,20240818,20240818,2.86,2.85,-0.01,iDEAL,NL56DEUT0265186420,NS Reizigers B.V.,VCR2083012768423734 8030609730868059 R20-83012...,NOTPROVIDED
4,136006396,EUR,20240818,20240818,2.85,9.85,7.00,SEPA OVERBOEKING,NL93BUNQ2036449662,Giedrius Mirklys,Transfer,NOTPROVIDED
5,136006396,EUR,20240819,20240819,9.85,1009.85,1000.00,SEPA OVERBOEKING,LT637300010150874468,GIEDRIUS MIRKLYS,TRANSFER TO MY DUTCH ACCOUNT,NOTPROVIDED
6,136006396,EUR,20240819,20240819,1009.85,986.50,-23.35,iDEAL,NL13ABNA0506417344,Espinoza,Jij dwong me,NOTPROVIDED
7,136006396,EUR,20240819,20240819,986.50,641.50,-345.00,SEPA OVERBOEKING,NL15RABO0105104329,Real Estate Malden B.V.,Huur september 2024,NOTPROVIDED
8,136006396,EUR,20240819,20240819,641.50,1141.50,500.00,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED,NOTPROVIDED
9,136006396,EUR,20240819,20240819,1141.50,1141.88,0.38,SEPA OVERBOEKING,NL93BUNQ2036449662,Giedrius Mirklys,NOTPROVIDED,NOTPROVIDED
